# Layoff Risk ETL & Feature Engineering (tech_layoffs.csv)
Notebook lives in `Data/`; all paths are relative to this notebook's folder.


## Prereqs
Source files in this folder (`Data/`):
- `tech_layoffs.csv` (raw Kaggle tech layoffs).
- Optional: `news_features.csv` (news counts/sentiment keyed by company).
- Optional: `finance_features.csv` (stock/market stats keyed by company).
Outputs (same folder): `cleaned_dataset.csv` (with target) and prediction input by dropping `target_high_risk`.


In [3]:
import pandas as pd
from pathlib import Path
import numpy as np

NOTEBOOK_DIR = Path(__file__).resolve().parent if '__file__' in globals() else Path().resolve()
DATA_DIR = NOTEBOOK_DIR
RAW_PATH = DATA_DIR / 'tech_layoffs.csv'
NEWS_PATH = DATA_DIR / 'news_features.csv'  # optional
FIN_PATH = DATA_DIR / 'finance_features.csv'  # optional
OUTPUT_CLEAN = DATA_DIR / 'cleaned_dataset.csv'
OUTPUT_PRED = DATA_DIR / 'your_dataset.csv'  # create on-the-fly if needed

LABEL_RULE = 'ratio'  # 'ratio' uses layoff_ratio, 'recent' uses past-12m activity
RATIO_THRESHOLD = 0.05  # 5% layoffs / total_employees estimate
RECENT_WINDOW_DAYS = 365


In [4]:
def parse_int(series):
    cleaned = series.astype(str).str.replace(',', '', regex=False)
    digits = cleaned.str.extract(r'([0-9]+)')[0]
    return pd.to_numeric(digits, errors='coerce')

def parse_pct(series):
    cleaned = series.astype(str).str.replace('%', '', regex=False).str.lower()
    cleaned = cleaned.replace({'unclear': np.nan})
    digits = cleaned.str.extract(r'([0-9]+\.?[0-9]*)')[0]
    return pd.to_numeric(digits, errors='coerce') / 100

def rolling_sum_days(group, days):
    s = group.set_index('date')['layoffs'].rolling(f'{days}D').sum()
    s.index = group.index
    return s

today = pd.Timestamp.today().normalize()

df = pd.read_csv(RAW_PATH)
print('Raw shape:', df.shape)
df.head()


Raw shape: (489, 9)


,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes
0,C2FO,20,2,12/9/2022,"Fintech, payments","Leawood, KS",Kansas City Business Journal,Private,NaN
1,VideoAmp,Unclear,2,9/19/2022,Advertising platforms,Los Angeles,Company memo,Private,NaN
2,Amperity,13,3,8/20/2022,"Saas, analytics",Seattle,Geekwire,Private,NaN
3,Addepar,20,3,1/18/2023,"fintech, data analytics","Mountain View, CA",Fortune,Private,NaN
4,Thirty Madison,24,3,5/13/2022,Health Care,New York,Axios,Private,"Per Axios, 22 of the employees laid off were f..."


In [5]:
# 1) Normalize text and dates
for col in ['company', 'industry', 'headquarter_location', 'status']:
    df[col] = df[col].str.lower().str.replace(r'[^a-z0-9 ,]', '', regex=True).str.strip()

df['date'] = pd.to_datetime(df['reported_date'])

# Keep recent 3 years
df = df[df['date'] >= today - pd.Timedelta(days=3*365)]

# Parse numeric fields
df['layoffs'] = parse_int(df['total_layoffs'])
df['impact_pct'] = parse_pct(df['impacted_workforce_percentage'])

# Avoid division by zero
df.loc[df['impact_pct'] == 0, 'impact_pct'] = np.nan
df['total_employees_est'] = df['layoffs'] / df['impact_pct']
df.head()


,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes,date,layoffs,impact_pct,total_employees_est
0,c2fo,20,2,12/9/2022,"fintech, payments","leawood, ks",Kansas City Business Journal,private,NaN,2022-12-09,20.0,0.02,1000.000000
3,addepar,20,3,1/18/2023,"fintech, data analytics","mountain view, ca",Fortune,private,NaN,2023-01-18,20.0,0.03,666.666667
6,armis security,25,4,12/8/2022,cybersecurity,san francisco,Calcalistech,private,NaN,2022-12-08,25.0,0.04,625.000000
8,headspace health,50,4,12/14/2022,"telehealth, wellness",los angeles,Bloomberg,private,NaN,2022-12-14,50.0,0.04,1250.000000
21,coindcx,100,6,1/11/2023,"blockchain, cryptocurrency","mumbai, india",Biometric Update,private,NaN,2023-01-11,100.0,0.06,1666.666667


In [6]:
# Optional joins
if NEWS_PATH.exists():
    news = pd.read_csv(NEWS_PATH)
    df = df.merge(news, on='company', how='left')
if FIN_PATH.exists():
    fin = pd.read_csv(FIN_PATH)
    df = df.merge(fin, on='company', how='left')
df.head()


,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes,date,layoffs,impact_pct,total_employees_est
0,c2fo,20,2,12/9/2022,"fintech, payments","leawood, ks",Kansas City Business Journal,private,NaN,2022-12-09,20.0,0.02,1000.000000
3,addepar,20,3,1/18/2023,"fintech, data analytics","mountain view, ca",Fortune,private,NaN,2023-01-18,20.0,0.03,666.666667
6,armis security,25,4,12/8/2022,cybersecurity,san francisco,Calcalistech,private,NaN,2022-12-08,25.0,0.04,625.000000
8,headspace health,50,4,12/14/2022,"telehealth, wellness",los angeles,Bloomberg,private,NaN,2022-12-14,50.0,0.04,1250.000000
21,coindcx,100,6,1/11/2023,"blockchain, cryptocurrency","mumbai, india",Biometric Update,private,NaN,2023-01-11,100.0,0.06,1666.666667


In [7]:
# 2) Aggregate per company with rolling windows (time-based)
df = df.sort_values(['company', 'date'])

df['layoffs_12m'] = df.groupby('company', group_keys=False).apply(lambda g: rolling_sum_days(g[['date', 'layoffs']], 365))
df['layoffs_last90d'] = df.groupby('company', group_keys=False).apply(lambda g: rolling_sum_days(g[['date', 'layoffs']], 90))
last_dates = df.groupby('company')['date'].transform('max')
df['days_since_last_layoff'] = (today - last_dates).dt.days

latest = df.sort_values('date').groupby('company').tail(1).copy()

# Fill estimated totals with industry median to avoid divide-by-zero
latest['total_employees_est'] = latest['total_employees_est'].replace([np.inf, -np.inf], np.nan)
latest['total_employees_est'] = latest['total_employees_est'].fillna(latest.groupby('industry')['total_employees_est'].transform('median'))
latest['total_employees_est'] = latest['total_employees_est'].fillna(latest['total_employees_est'].median())

latest['layoff_ratio'] = latest['layoffs_12m'] / latest['total_employees_est']
latest['impacted_pct_recent'] = latest['impact_pct']

latest.head()


C:\Users\Daniel\AppData\Local\Temp\ipykernel_49616\1212669479.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['layoffs_12m'] = df.groupby('company', group_keys=False).apply(lambda g: rolling_sum_days(g[['date', 'layoffs']], 365))
C:\Users\Daniel\AppData\Local\Temp\ipykernel_49616\1212669479.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['layoffs_last90d'] = df.groupby('company', group_keys=False).apply

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes,date,layoffs,impact_pct,total_employees_est,layoffs_12m,layoffs_last90d,days_since_last_layoff,layoff_ratio,impacted_pct_recent
245,glorifi,Unclear,100,11/21/2022,fintech,dallas,WSJ,private,NaN,2022-11-21,NaN,1.00,1209.090909,NaN,NaN,1094,NaN,1.00
244,assure,Unclear,100,11/23/2022,fintech,salt lake city,Axios,private,NaN,2022-11-23,NaN,1.00,1209.090909,NaN,NaN,1092,NaN,1.00
219,ncx,Unclear,40,11/28/2022,"renewable energy, forestry",san francisco,NCX,private,NaN,2022-11-28,NaN,0.40,1209.090909,NaN,NaN,1087,NaN,0.40
254,blockfi,Unclear,100,11/28/2022,crypto,jersey city,Decrypt,private,NaN,2022-11-28,NaN,1.00,4906.976744,NaN,NaN,1087,NaN,1.00
203,candy digital,33,33,11/28/2022,cryptocurrency,new york,Decrypt,private,NaN,2022-11-28,33.0,0.33,100.000000,33.0,33.0,1087,0.33,0.33


In [8]:
# 3) Create target label
if LABEL_RULE == 'ratio':
    ratio_proxy = latest['layoff_ratio'].fillna(latest['impacted_pct_recent'])
    latest['target_high_risk'] = (ratio_proxy >= RATIO_THRESHOLD).astype(int)
else:
    cutoff = today - pd.Timedelta(days=RECENT_WINDOW_DAYS)
    recent_any = (df[df['date'] >= cutoff]
                  .groupby('company')['layoffs']
                  .sum() > 0)
    latest = latest.merge(recent_any.rename('recent_flag'), left_on='company', right_index=True, how='left')
    latest['target_high_risk'] = latest['recent_flag'].fillna(False).astype(int)

print(latest['target_high_risk'].value_counts())


target_high_risk
1    115
0     27
Name: count, dtype: int64


In [9]:
# 4) Select final columns and export
feature_cols = [
    'company', 'industry', 'headquarter_location', 'status',
    'layoffs_12m', 'layoffs_last90d', 'days_since_last_layoff',
    'total_employees_est', 'layoff_ratio', 'impacted_pct_recent'
]
feature_cols = [c for c in feature_cols if c in latest.columns]

clean = latest[feature_cols + ['target_high_risk']].copy()
clean = clean.dropna(subset=['target_high_risk'])

for col in feature_cols:
    if clean[col].dtype.kind in 'biufc':
        clean[col] = clean[col].fillna(clean[col].median())
    else:
        clean[col] = clean[col].fillna('unknown')

clean.to_csv(OUTPUT_CLEAN, index=False)
predict_template = clean.drop(columns=['target_high_risk'])
predict_template.to_csv(OUTPUT_PRED, index=False)

clean.head(), predict_template.head()


(           company                    industry headquarter_location   status  \
 245        glorifi                     fintech               dallas  private   
 244         assure                     fintech       salt lake city  private   
 219            ncx  renewable energy, forestry        san francisco  private   
 254        blockfi                      crypto          jersey city  private   
 203  candy digital              cryptocurrency             new york  private   
 
      layoffs_12m  layoffs_last90d  days_since_last_layoff  \
 245        100.0            100.0                    1094   
 244        100.0            100.0                    1092   
 219        100.0            100.0                    1087   
 254        100.0            100.0                    1087   
 203         33.0             33.0                    1087   
 
      total_employees_est  layoff_ratio  impacted_pct_recent  target_high_risk  
 245          1209.090909          0.12                 1

### Next steps
- Inspect class balance; adjust `RATIO_THRESHOLD` if classes are too imbalanced.
- In modeling notebook, encode categoricals (industry, headquarter_location, status) then train DecisionTree/RandomForest/XGBoost.
- Save the trained model to `Model/` and align Streamlit/Notebook prediction input with `cleaned_dataset.csv` columns minus `target_high_risk`.
